# Classification using neural networks

### Author
- William Fitzjohn

### Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim
from torchvision import datasets,transforms
# Used if running in Jupyter
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Importing Data

In [2]:
# Inspired by code from 'https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html'
# Function used to normalize downloaded data
transform_function = transforms.Compose([\
    transforms.ToTensor(),\
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]\
)

# Setting batch size
batch_size = 32

# Creating training set
train_set = datasets.CIFAR10(\
    root='./data',\
    train=True,\
    download=True,\
    transform=transform_function\
)
train_loader = torch.utils.data.DataLoader(
    train_set,\
    batch_size=batch_size,\
    shuffle=True,\
    num_workers=2\
)

# Creating testing set
test_set = datasets.CIFAR10(
    root='./data',\
    train=False,\
    download=True,\
    transform=transform_function\
)
test_loader = torch.utils.data.DataLoader(
    test_set,\
    batch_size=batch_size,\
    shuffle=False,\
    num_workers=2\
)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Setting Classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Setting Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# [Part I] Fully Connected Neural Network

### Creating & Initializing Network

In [4]:
# Hyperparameters For Training Environment
input_size = 32*32*3 #32px x 32px x RGB
num_classes = len(classes)
learning_rate = 0.003
epochs = 3

In [5]:
# Inspired by code from 'https://aladdinpersson.medium.com/pytorch-neural-network-tutorial-7e871d6be7c4'

class FullNN(nn.Module):
    # Initializing 3 layer fully connected NN
    def __init__(self, i_size, n_classes):
        super().__init__()
        self.fc1 = nn.Linear(i_size, round(i_size/5))
        self.fc2 = nn.Linear(round(i_size/5), round(i_size/25))
        self.fc3 = nn.Linear(round(i_size/25), n_classes)

    # Sending data to NN
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = functional.relu(self.fc1(x))
        x = functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Initializing Model
fnn_model = FullNN(input_size, num_classes).to(device)

In [7]:
# Create Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(fnn_model.parameters(), lr=learning_rate, momentum=0.9)

### Training Network

In [8]:
for epoch in range(epochs):
    for i, (data, targets) in enumerate(train_loader):
        # Setup GPU integration
        data = data.to(device=device)
        targets = targets.to(device=device)

        data = data.reshape(data.shape[0], -1) # Flatten Data

        # Forward propagation
        scores = fnn_model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad() # Reset old gradients
        loss.backward() # Back progpagate
        optimizer.step() # Gradient descent

### Testing Network

In [9]:
# Inspired by code from 'https://youtu.be/Jy4wM2X21u0'
def check_accuracy_full(loader, model):
    # Counters
    num_correct = 0
    num_samples = 0
    model.eval() #Evaluate mode

    # Disable gradients for faster computation
    with torch.no_grad():
        for x, y in loader:
            # Formatting data
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            # Scoring data
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        return [num_correct, num_samples, float(num_correct) / float(num_samples) * 100]

    model.train() #Train mode

In [10]:
[f_train_correct, f_train_samples, f_train_accuracy] = check_accuracy_full(train_loader, fnn_model)
[f_test_correct, f_test_samples, f_test_accuracy] = check_accuracy_full(test_loader, fnn_model)

In [11]:
# Fully Connected Neural Net
print("[Fully Connected Neural Net]\n"+\
     "Train Set: "+str(int(f_train_correct))+"/"+str(f_train_samples)+" =",f"{f_train_accuracy: .2f}","% Accurate\n" +\
     "Test Set: "+str(int(f_test_correct))+"/"+str(f_test_samples)+" =",f"{f_test_accuracy: .2f}","% Accurate\n")

[Fully Connected Neural Net]
Train Set: 28111/50000 =  56.22 % Accurate
Test Set: 5145/10000 =  51.45 % Accurate



# [Part II] Convolutional Neural Network

### Creating & Initializing

In [12]:
# Hyperparameters For Training Environment
learning_rate = 0.006
epochs = 5
momentum = 0.9

In [13]:
# Inspired by code from 'https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html'
# and 'https://pyimagesearch.com/2021/07/19/pytorch-training-your-first-convolutional-neural-network-cnn/'

class ConvNN(nn.Module):
    def __init__(self, i_size, n_classes):
        super().__init__()
        # 2 Layers of conv, 2 layers of FC
        self.conv1 = nn.Conv2d(3, 20, 5) # 3 channels in, 20 out, 5 kernal size 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 50, 5) # 20 channels in, 50 out, 5 kernal size 
        self.fc1 = nn.Linear(1250, 500) # 1250 in, 500 out
        self.fc2 = nn.Linear(500, n_classes) # 500 in, classes out

    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x))) # conv, relu, pool
        x = self.pool(functional.relu(self.conv2(x))) # conv, relu, pool
        x = torch.flatten(x, 1) # flatten
        x = functional.relu(self.fc1(x)) # fc, relu
        x = self.fc2(x) # fc
        return x

In [14]:
# Initializing Model
cnn_model = ConvNN(input_size, num_classes).to(device)
# Create Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn_model.parameters(), lr=learning_rate, momentum=momentum)

### Training Network

In [15]:
for epoch in range(epochs):
    for i, (data, targets) in enumerate(train_loader):
        # Setup GPU integration
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward progpagate
        scores = cnn_model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad() # Reset old gradients
        loss.backward() # Back progpagate
        optimizer.step() # Gradient descent

### Testing Network

In [16]:
# Inspired by code from 'https://youtu.be/Jy4wM2X21u0'
def check_accuracy_conv(loader, model):
    # Counters
    num_correct = 0
    num_samples = 0
    model.eval() #Evaluate mode

    # Disable gradients for faster computation
    with torch.no_grad():
        for x, y in loader:
            # Formatting data
            x = x.to(device=device)
            y = y.to(device=device)

            # Scoring data
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        return [num_correct, num_samples, float(num_correct) / float(num_samples) * 100]
    model.train() #Train mode

In [17]:
[c_train_correct, c_train_samples, c_train_accuracy] = check_accuracy_conv(train_loader, cnn_model)
[c_test_correct, c_test_samples, c_test_accuracy] = check_accuracy_conv(test_loader, cnn_model)

# Results

In [18]:
# Fully Connected Neural Net
print("[Fully Connected Neural Net]\n"+\
     "Train Set: "+str(int(f_train_correct))+"/"+str(f_train_samples)+" =",f"{f_train_accuracy: .2f}","% Accurate\n" +\
     "Test Set: "+str(int(f_test_correct))+"/"+str(f_test_samples)+" =",f"{f_test_accuracy: .2f}","% Accurate\n")
# Convolutional Neural Net
print("[Convolutional Neural Net]\n"+\
     "Train Set: "+str(int(c_train_correct))+"/"+str(c_train_samples)+" =",f"{c_train_accuracy: .2f}","% Accurate\n" +\
     "Test Set: "+str(int(c_test_correct))+"/"+str(c_test_samples)+" =",f"{c_test_accuracy: .2f}","% Accurate")

[Fully Connected Neural Net]
Train Set: 28111/50000 =  56.22 % Accurate
Test Set: 5145/10000 =  51.45 % Accurate

[Convolutional Neural Net]
Train Set: 40252/50000 =  80.50 % Accurate
Test Set: 6845/10000 =  68.45 % Accurate
